In [6]:
import numpy
from matplotlib import pyplot as plt
import PySpin
from datetime import datetime
import os

In [7]:
def configure_custom_image_settings(cam,
    num_images,
    downsize_width,
    downsize_height,
    acquisition_frame_rate, 
    exposure_time_to_set,
    gain_to_set):
    """
    Configures the width and height of the image, the
    offset x and y, the acquisition frame rate, exposure, and gain. 

    :param cam: Camera to configure settings on.
    :param downsize_width: default=2, max width (5472) divided by downsize_width
    :param downsize_height: default=2, max height (3648) divided by downsize_height
    :param acquisition_frame_rate: default = 3 Hz, min 2.33, max depends on width/height capture size and connection speed
    :param exposure_time_to_set: default = 15000 microseconds
    :param gain_to_set: default = 6.29
    :type cam: CameraPtr
    :return: True if successful, False otherwise.
    :rtype: bool
    """
    print('\n*** CONFIGURING CUSTOM IMAGE SETTINGS ***\n')
    max_width = 5472
    max_height = 3648
    set_width = int(max_width/downsize_width)
    set_height = int(max_height/downsize_height)
    set_offset_x = int((max_width-set_width)/2)
    set_offset_y = int((max_height-set_height)/2)
    acquisition_frame_rate = acquisition_frame_rate #frames per second
    exposure_time_to_set = exposure_time_to_set #milliseconds
    gain_to_set = gain_to_set
      
    try:
        result = True
        
        #WIDTH
        if cam.Width.GetAccessMode() == PySpin.RW and cam.Width.GetInc() != 0 and cam.Width.GetMax != 0:
            cam.Width.SetValue(set_width)
            print('Width set to %i...' % cam.Width.GetValue())
        else:
            print('Width not available...')
            result = False
        
        #HEIGHT
        if cam.Height.GetAccessMode() == PySpin.RW and cam.Height.GetInc() != 0 and cam.Height.GetMax != 0:
            cam.Height.SetValue(set_height)
            print('Height set to %i...' % cam.Height.GetValue())
        else:
            print('Height not available...')
            result = False

        #OFFSET X
        if cam.OffsetX.GetAccessMode() == PySpin.RW:
            cam.OffsetX.SetValue(set_offset_x)
            print('Offset X set to %d...' % cam.OffsetX.GetValue())

        else:
            print('Offset X not available...')
            result = False
        
        #OFFSET Y
        if cam.OffsetY.GetAccessMode() == PySpin.RW:
            cam.OffsetY.SetValue(set_offset_y)
            print('Offset Y set to %d...' % cam.OffsetY.GetValue())

        else:
            print('Offset Y not available...')
            result = False
        
        #EXPOSURE
        if cam.ExposureAuto.GetAccessMode() != PySpin.RW:
            print('Unable to disable automatic exposure. Aborting...')
            return False

        cam.ExposureAuto.SetValue(PySpin.ExposureAuto_Off)
        print('Automatic exposure disabled...')

        if cam.ExposureTime.GetAccessMode() != PySpin.RW:
            print('Unable to set exposure time. Aborting...')
            return False

        # Ensure desired exposure time does not exceed the maximum
        exposure_time_to_set = min(cam.ExposureTime.GetMax(), exposure_time_to_set)
        cam.ExposureTime.SetValue(exposure_time_to_set)
        
        #GAIN
        if cam.GainAuto.GetAccessMode() != PySpin.RW:
            print('Unable to disable automatic gain. Aborting...')
            return False

        cam.GainAuto.SetValue(PySpin.GainAuto_Off)
        print('Automatic gain disabled...')

        if cam.Gain.GetAccessMode() != PySpin.RW:
            print('Unable to set exposure time. Aborting...')
            return False

        # Ensure desired gain does not exceed the maximum
        gain_to_set = min(cam.Gain.GetMax(), gain_to_set)
        cam.Gain.SetValue(gain_to_set)
        
        #FRAME RATE
        cam.AcquisitionFrameRateEnable.SetValue(True)
        cam.AcquisitionFrameRate.SetValue(acquisition_frame_rate)    
        print("Acquisition frame rate set to %3.4f Hz..." % cam.AcquisitionFrameRate.GetValue())        
        
    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        return False

    return result

def configure_chunk_data(nodemap):
    """
    This function configures the camera to add chunk data to each image. It does
    this by enabling each type of chunk data before enabling chunk data mode.
    When chunk data is turned on, the data is made available in both the nodemap
    and each image.

    :param nodemap: Transport layer device nodemap.
    :type nodemap: INodeMap
    :return: True if successful, False otherwise
    :rtype: bool
    """
    try:
        result = True
        print('\n*** CONFIGURING CHUNK DATA ***\n')

        # Activate chunk mode
        chunk_mode_active = PySpin.CBooleanPtr(nodemap.GetNode('ChunkModeActive'))

        if PySpin.IsAvailable(chunk_mode_active) and PySpin.IsWritable(chunk_mode_active):
            chunk_mode_active.SetValue(True)

        print('Chunk mode activated...')

        # Enable all types of chunk data
        chunk_selector = PySpin.CEnumerationPtr(nodemap.GetNode('ChunkSelector'))

        if not PySpin.IsAvailable(chunk_selector) or not PySpin.IsReadable(chunk_selector):
            print('Unable to retrieve chunk selector. Aborting...\n')
            return False

        # Retrieve entries
        entries = [PySpin.CEnumEntryPtr(chunk_selector_entry) for chunk_selector_entry in chunk_selector.GetEntries()]

        print('Enabling entries...')

        # Iterate through our list and select each entry node to enable
        for chunk_selector_entry in entries:
            # Go to next node if problem occurs
            if not PySpin.IsAvailable(chunk_selector_entry) or not PySpin.IsReadable(chunk_selector_entry):
                continue

            chunk_selector.SetIntValue(chunk_selector_entry.GetValue())

            chunk_str = '\t {}:'.format(chunk_selector_entry.GetSymbolic())

            # Retrieve corresponding boolean
            chunk_enable = PySpin.CBooleanPtr(nodemap.GetNode('ChunkEnable'))

            # Enable the boolean, thus enabling the corresponding chunk data
            if not PySpin.IsAvailable(chunk_enable):
                print('{} not available'.format(chunk_str))
                result = False
            elif chunk_enable.GetValue() is True:
                print('{} enabled'.format(chunk_str))
            elif PySpin.IsWritable(chunk_enable):
                chunk_enable.SetValue(True)
                print('{} enabled'.format(chunk_str))
            else:
                print('{} not writable'.format(chunk_str))
                result = False

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        result = False

    return result

def reset_exposure(cam):
    """
    This function returns the camera to a normal state by re-enabling automatic exposure.

    :param cam: Camera to reset exposure on.
    :type cam: CameraPtr
    :return: True if successful, False otherwise.
    :rtype: bool
    """
    try:
        result = True

        # Turn automatic exposure back on
        #
        # *** NOTES ***
        # Automatic exposure is turned on in order to return the camera to its
        # default state.

        if cam.ExposureAuto.GetAccessMode() != PySpin.RW:
            print('Unable to enable automatic exposure (node retrieval). Non-fatal error...')
            return False

        cam.ExposureAuto.SetValue(PySpin.ExposureAuto_Continuous)

        print('Automatic exposure enabled...')
        
        if cam.GainAuto.GetAccessMode() != PySpin.RW:
            print('Unable to disable automatic gain. Aborting...')
            return False
        
        cam.GainAuto.SetValue(PySpin.GainAuto_Continuous)
        
        print('Automatic gain enabled...')
        
    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        result = False

    return result

def print_device_info(cam):
    """
    This function prints the device information of the camera from the transport
    layer; please see NodeMapInfo example for more in-depth comments on printing
    device information from the nodemap.

    :param cam: Camera to get device information from.
    :type cam: CameraPtr
    :return: True if successful, False otherwise.
    :rtype: bool
    """

    print('\n*** DEVICE INFORMATION ***\n')

    try:
        result = True
        nodemap = cam.GetTLDeviceNodeMap()

        node_device_information = PySpin.CCategoryPtr(nodemap.GetNode('DeviceInformation'))

        if PySpin.IsAvailable(node_device_information) and PySpin.IsReadable(node_device_information):
            features = node_device_information.GetFeatures()
            for feature in features:
                node_feature = PySpin.CValuePtr(feature)
                print('%s: %s' % (node_feature.GetName(),
                                  node_feature.ToString() if PySpin.IsReadable(node_feature) else 'Node not readable'))

        else:
            print('Device control information not available.')

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex.message)
        return False

    return result

def check_or_create_dir(dirs, cwd=os.getcwd()):
    """
    This function iterates through a list of strings and creates nested directories with the string names if they don't already exist

    :param dirs: list of strings to created nested directories in sequential order \
    :param cwd: current working directory, default = os.getcwd() 
    """
    if len(dirs) > 0:
        if not os.path.exists(dirs[0]):
            os.mkdir(dirs[0])
            check_or_create_dir(dirs, cwd)
        else:
            os.chdir(dirs[0])
            dirs.pop(0)
            check_or_create_dir(dirs,cwd)
    else:
        os.chdir(cwd)
        
        
def acquire_images(cam, sample, num_images):
    """
    This function acquires and saves user-defined number of images from a device; please see
    Acquisition example for more in-depth comments on the acquisition of images.

    :param cam: Camera to acquire images from.
    :param sample: sample name for creating directories and filename for saving
    :param num_iamges: number of images to acquire
    :type cam: CameraPtr
    :return: True if successful, False otherwise.
    :rtype: bool
    """
    print('\n*** IMAGE ACQUISITION ***\n')

    try:
        result = True

        # Set acquisition mode to continuous
        if cam.AcquisitionMode.GetAccessMode() != PySpin.RW:
            print('Unable to set acquisition mode to continuous. Aborting...')
            return False

        cam.AcquisitionMode.SetValue(PySpin.AcquisitionMode_Continuous)
        print('Acquisition mode set to continuous...')

        # Begin acquiring images
        cam.BeginAcquisition()

        print('Acquiring images...')

        # Get device serial number for filename
        device_serial_number = ''
        if cam.TLDevice.DeviceSerialNumber is not None and cam.TLDevice.DeviceSerialNumber.GetAccessMode() == PySpin.RO:
            device_serial_number = cam.TLDevice.DeviceSerialNumber.GetValue()

            print('Device serial number retrieved as %s...' % device_serial_number)

        # Retrieve, convert, and save images
        for i in range(num_images):

            try:
                # Retrieve next received image and ensure image completion
                image_result = cam.GetNextImage()

                if image_result.IsIncomplete():
                    print('Image incomplete with image status %d...' % image_result.GetImageStatus())

                else:
                    # Print image information
                    width = image_result.GetWidth()
                    height = image_result.GetHeight()
                    print('Grabbed Image %d, width = %d, height = %d' % (i, width, height))

                    # Convert image to Mono8
                    #image_converted = image_result.Convert(PySpin.PixelFormat_Mono8)
                    chunk_data = image_result.GetChunkData()
                    
                    # Retrieve frame ID
                    frame_id = chunk_data.GetFrameID()
                    # Retrieve timestamp
                    timestamp = chunk_data.GetTimestamp()
                    #today's date
                    today_date = datetime.date(datetime.today()).isoformat()
                    #acquisition_frame_rate
                    acq_frame_rate = str(int(round(cam.AcquisitionFrameRate.GetValue())))+'Hz'
                    
                    
                    # Create a unique filename
                    if chunk_data:
                        filename = '%s_%s_%s_%s_%04d.tif' % (today_date,str(sample),timestamp,acq_frame_rate, i)
                    
                    abs_path = os.path.join(os.getcwd(),sample.replace('_', '\\'),filename)
                    
                    # Save image
                    TIFF_format = PySpin.TIFFOption()
                    TIFF_format.compression = PySpin.TIFFOption.NONE
                    image_result.Save(abs_path, TIFF_format)

                    print('Image saved at %s' % filename)

                # Release image
                image_result.Release()

            except PySpin.SpinnakerException as ex:
                print('Error: %s' % ex)
                result = False

        # End acquisition
        cam.EndAcquisition()

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        result = False

    return result

def run_single_camera(cam, 
    sample,
    num_images,
    downsize_width,
    downsize_height,
    acquisition_frame_rate, 
    exposure_time_to_set,
    gain_to_set):
    """
     This function acts as the body of the example; please see NodeMapInfo_QuickSpin example for more
     in-depth comments on setting up cameras.

     :param cam: Camera to run example on.
     :type cam: CameraPtr
     :return: True if successful, False otherwise.
     :rtype: bool
    """
    try:
        # Initialize camera
        cam.Init()

        # Print device info
        result = print_device_info(cam)

        # Configure camera settings
        if not configure_custom_image_settings(cam,
            num_images,
            downsize_width,
            downsize_height,
            acquisition_frame_rate, 
            exposure_time_to_set,
            gain_to_set):
            return False
        
        # Retrieve GenICam nodemap
        nodemap = cam.GetNodeMap()
        
        # Configure chunk data
        if configure_chunk_data(nodemap) is False:
            return False
        
        # Acquire images
        result &= acquire_images(cam=cam, sample=sample, num_images=num_images)
        
        #reset exposure modes
        reset_exposure(cam)
        # Deinitialize camera
        cam.DeInit()

        return result

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        return False

In [8]:
def begin_capture(sample,
    num_images = 10,
    downsize_width = 2,
    downsize_height = 2,
    acquisition_frame_rate = 3, 
    exposure_time_to_set = 15000.0,
    gain_to_set = 6.29):
    
    result = True
    # Get system
    system = PySpin.System.GetInstance()
    # Get camera list
    cam_list = system.GetCameras()
    num_cameras = cam_list.GetSize()

    print('Number of cameras detected: %d' % num_cameras)
    # Finish if there are no cameras
    if num_cameras == 0:
        # Clear camera list before releasing system
        cam_list.Clear()

        # Release system instance
        system.ReleaseInstance()

        print('Not enough cameras!')
        return False
    
    check_or_create_dir(sample.split('_'), cwd=os.getcwd())
    
    cam = cam_list.GetByIndex(0)
    result &= run_single_camera(cam, 
                sample,
                num_images,
                downsize_width,
                downsize_height,
                acquisition_frame_rate, 
                exposure_time_to_set,
                gain_to_set)
    
    del cam
    cam_list.Clear()
    system.ReleaseInstance()
    print("Done!")
    return result

In [4]:
#RUN USB CAMERA

In [9]:
capture_time = 25 # minutes
capture_time = capture_time * 60 #seconds
acquisition_rate = 2 #Hz
num_images = capture_time*acquisition_rate
# num_images=10 #OVERRIDE FOR TESTING
sample_name = 'DAY14_SFMPLUS_B2' # underscore will be divided into sub-directories when saving e.g. 'SFM_DAY4_A1' will be saved in folder SFM\\DAY4\\A1
downsize_width = 6
downsize_height = 1
# exposure_time_to_set
# gain_to_set

In [10]:
#BEGIN CAPTURE
begin_capture(sample=sample_name, num_images=num_images, acquisition_frame_rate=acquisition_rate, downsize_width=downsize_width, downsize_height=downsize_height)

Number of cameras detected: 1

*** DEVICE INFORMATION ***

DeviceID: 18474805
DeviceSerialNumber: 18474805
DeviceVendorName: FLIR
DeviceModelName: Blackfly S BFS-U3-200S6C
DeviceType: U3V
DeviceDisplayName: FLIR Blackfly S BFS-U3-200S6C
DeviceAccessStatus: ReadWrite
DeviceVersion: 1705.0.124.0
DeviceDriverVersion: PGRUSBCam3.sys : 2.7.3.249
DeviceUserID: 
DeviceIsUpdater: 0
DeviceInstanceId: USB\VID_1E10&PID_4000&MI_00\8&356F58B8&0&0000
DeviceCurrentSpeed: SuperSpeed
GUIXMLLocation: Device
GUIXMLPath: Input.xml
GenICamXMLLocation: Device
GenICamXMLPath: 
DeviceU3VProtocol: 1

*** CONFIGURING CUSTOM IMAGE SETTINGS ***

Width set to 912...
Height set to 3648...
Offset X set to 2280...
Offset Y set to 0...
Automatic exposure disabled...
Automatic gain disabled...
Acquisition frame rate set to 2.0000 Hz...

*** CONFIGURING CHUNK DATA ***

Chunk mode activated...
Enabling entries...
	 Image: enabled
	 CRC: enabled
	 FrameID: enabled
	 OffsetX: enabled
	 OffsetY: enabled
	 Width: enabled
	 H

False

In [ ]:
%run Examples\\Python3\\ChunkData.py

Library version: 1.19.0.22
Number of cameras detected: 1
Running example for camera 0...

*** DEVICE INFORMATION ***

DeviceID: 18474805
DeviceSerialNumber: 18474805
DeviceVendorName: FLIR
DeviceModelName: Blackfly S BFS-U3-200S6C
DeviceType: U3V
DeviceDisplayName: FLIR Blackfly S BFS-U3-200S6C
DeviceAccessStatus: ReadWrite
DeviceVersion: 1705.0.124.0
DeviceDriverVersion: PGRUSBCam3.sys : 2.7.3.249
DeviceUserID: 
DeviceIsUpdater: 0
DeviceInstanceId: USB\VID_1E10&PID_4000&MI_00\6&356F58B8&0&0000
DeviceCurrentSpeed: HighSpeed
GUIXMLLocation: Device
GUIXMLPath: Input.xml
GenICamXMLLocation: Device
GenICamXMLPath: 
DeviceU3VProtocol: 1

*** CONFIGURING CHUNK DATA ***

Chunk mode activated...
Enabling entries...
	 Image: enabled
	 CRC: enabled
	 FrameID: enabled
	 OffsetX: enabled
	 OffsetY: enabled
	 Width: enabled
	 Height: enabled
	 ExposureTime: enabled
	 Gain: enabled
	 BlackLevel: enabled
	 PixelFormat: enabled
	 Timestamp: enabled
	 SequencerSetActive: enabled
	 SerialData: enabled
	

Done! Press Enter to exit... 


In [7]:
system = PySpin.System.GetInstance()
# Get camera list
cam_list = system.GetCameras()
num_cameras = cam_list.GetSize()

print('Number of cameras detected: %d' % num_cameras)
# Finish if there are no cameras
if num_cameras == 0:
    # Clear camera list before releasing system
    cam_list.Clear()

    # Release system instance
    system.ReleaseInstance()

    print('Not enough cameras!')
    input('Done! Press Enter to exit...')


cam = cam_list.GetByIndex(0)
cam.Init()

Number of cameras detected: 1


In [10]:
PySpin.RW

4

In [9]:
cam.Gain.GetAccessMode()

3

In [49]:
cam.TLDevice.DeviceSerialNumber.GetValue()

'18474805'

In [50]:
cam.AcquisitionFrameRateEnable()

True

In [12]:
cam.DeInit()

In [51]:
cam.AcquisitionFrameRate()

3.0043462073974694

In [52]:
cam.GainAuto()

2

In [13]:
%run NodeMapInfo.py

Library version: 1.19.0.22
Number of cameras detected: 1
Running example for camera 0...

*** PRINTING TRANSPORT LAYER DEVICE NODEMAP *** 

Root
    Device Information
        DeviceID: 18474805
        Device Serial Number: 18474805
        Device Vendor Name  : FLIR
        Device Model Name: Blackfly S BFS-U3-200S6C
        Device Type: U3V
        Device Display Name: FLIR Blackfly S BFS-U3-200S6C
        Device Access Status: ReadWrite
        Device Version: 1705.0.124.0
        Device Driver Version: PGRUSBCam3.sys : 2.7.3.249
        Device User ID: 
        Device Is Updater Mode: False
        DeviceInstanceId: USB\VID_1E10&PID_4000&MI_00\6&356F5...
        Device Current Speed: HighSpeed
        GUI XML Source: Device
        GUI XML Path: Input.xml
        GenICam XML Source: Device
        GenICam XML Path: 
        Device Is In U3V Protocol: True

    Device Control
        Device Endianess Mechanism: Standard


*** PRINTING TL STREAM NODEMAP ***

Root
    Stream Informat

Done! Press Enter to exit... 


In [12]:
%run Examples\\Python3\\Enumeration.py

Library version: 1.19.0.22
Number of interfaces detected: 4
Number of cameras detected: 1

*** QUERYING INTERFACES ***

GEV Interface 0
	No devices detected.

USB Interface 0
	No devices detected.

USB Interface 1
	No devices detected.

USB Interface 2
	Device 0 FLIR Blackfly S BFS-U3-200S6C 



Done! Press Enter to exit... 


In [3]:
#ACQUISITION EXAMPLE
# coding=utf-8
# =============================================================================
#  Copyright © 2017 FLIR Integrated Imaging Solutions, Inc. All Rights Reserved.
#
#  This software is the confidential and proprietary information of FLIR
#  Integrated Imaging Solutions, Inc. ("Confidential Information"). You
#  shall not disclose such Confidential Information and shall use it only in
#  accordance with the terms of the license agreement you entered into
#  with FLIR Integrated Imaging Solutions, Inc. (FLIR).
#
#  FLIR MAKES NO REPRESENTATIONS OR WARRANTIES ABOUT THE SUITABILITY OF THE
#  SOFTWARE, EITHER EXPRESSED OR IMPLIED, INCLUDING, BUT NOT LIMITED TO, THE
#  IMPLIED WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR
#  PURPOSE, OR NON-INFRINGEMENT. FLIR SHALL NOT BE LIABLE FOR ANY DAMAGES
#  SUFFERED BY LICENSEE AS A RESULT OF USING, MODIFYING OR DISTRIBUTING
#  THIS SOFTWARE OR ITS DERIVATIVES.
# =============================================================================
#
# Acquisition.py shows how to acquire images. It relies on
# information provided in the Enumeration example. Also, check out the
# ExceptionHandling and NodeMapInfo examples if you haven't already.
# ExceptionHandling shows the handling of standard and Spinnaker exceptions
# while NodeMapInfo explores retrieving information from various node types.
#
# This example touches on the preparation and cleanup of a camera just before
# and just after the acquisition of images. Image retrieval and conversion,
# grabbing image data, and saving images are all covered as well.
#
# Once comfortable with Acquisition, we suggest checking out
# AcquisitionMultipleCamera, NodeMapCallback, or SaveToAvi.
# AcquisitionMultipleCamera demonstrates simultaneously acquiring images from
# a number of cameras, NodeMapCallback serves as a good introduction to
# programming with callbacks and events, and SaveToAvi exhibits video creation.

import os
import PySpin

NUM_IMAGES = 10  # number of images to grab


def acquire_images(cam, nodemap, nodemap_tldevice):
    """
    This function acquires and saves 10 images from a device.

    :param cam: Camera to acquire images from.
    :param nodemap: Device nodemap.
    :param nodemap_tldevice: Transport layer device nodemap.
    :type cam: CameraPtr
    :type nodemap: INodeMap
    :type nodemap_tldevice: INodeMap
    :return: True if successful, False otherwise.
    :rtype: bool
    """

    print('*** IMAGE ACQUISITION ***\n')
    try:
        result = True

        # Set acquisition mode to continuous
        #
        #  *** NOTES ***
        #  Because the example acquires and saves 10 images, setting acquisition
        #  mode to continuous lets the example finish. If set to single frame
        #  or multiframe (at a lower number of images), the example would just
        #  hang. This would happen because the example has been written to
        #  acquire 10 images while the camera would have been programmed to
        #  retrieve less than that.
        #
        #  Setting the value of an enumeration node is slightly more complicated
        #  than other node types. Two nodes must be retrieved: first, the
        #  enumeration node is retrieved from the nodemap; and second, the entry
        #  node is retrieved from the enumeration node. The integer value of the
        #  entry node is then set as the new value of the enumeration node.
        #
        #  Notice that both the enumeration and the entry nodes are checked for
        #  availability and readability/writability. Enumeration nodes are
        #  generally readable and writable whereas their entry nodes are only
        #  ever readable.
        #
        #  Retrieve enumeration node from nodemap

        # In order to access the node entries, they have to be casted to a pointer type (CEnumerationPtr here)
        node_acquisition_mode = PySpin.CEnumerationPtr(nodemap.GetNode('AcquisitionMode'))
        if not PySpin.IsAvailable(node_acquisition_mode) or not PySpin.IsWritable(node_acquisition_mode):
            print('Unable to set acquisition mode to continuous (enum retrieval). Aborting...')
            return False

        # Retrieve entry node from enumeration node
        node_acquisition_mode_continuous = node_acquisition_mode.GetEntryByName('Continuous')
        if not PySpin.IsAvailable(node_acquisition_mode_continuous) or not PySpin.IsReadable(node_acquisition_mode_continuous):
            print('Unable to set acquisition mode to continuous (entry retrieval). Aborting...')
            return False

        # Retrieve integer value from entry node
        acquisition_mode_continuous = node_acquisition_mode_continuous.GetValue()

        # Set integer value from entry node as new value of enumeration node
        node_acquisition_mode.SetIntValue(acquisition_mode_continuous)

        print('Acquisition mode set to continuous...')

        #  Begin acquiring images
        #
        #  *** NOTES ***
        #  What happens when the camera begins acquiring images depends on the
        #  acquisition mode. Single frame captures only a single image, multi
        #  frame catures a set number of images, and continuous captures a
        #  continuous stream of images. Because the example calls for the
        #  retrieval of 10 images, continuous mode has been set.
        #
        #  *** LATER ***
        #  Image acquisition must be ended when no more images are needed.
        cam.BeginAcquisition()

        print('Acquiring images...')

        #  Retrieve device serial number for filename
        #
        #  *** NOTES ***
        #  The device serial number is retrieved in order to keep cameras from
        #  overwriting one another. Grabbing image IDs could also accomplish
        #  this.
        device_serial_number = ''
        node_device_serial_number = PySpin.CStringPtr(nodemap_tldevice.GetNode('DeviceSerialNumber'))
        if PySpin.IsAvailable(node_device_serial_number) and PySpin.IsReadable(node_device_serial_number):
            device_serial_number = node_device_serial_number.GetValue()
            print('Device serial number retrieved as %s...' % device_serial_number)

        # Retrieve, convert, and save images
        for i in range(NUM_IMAGES):
            try:

                #  Retrieve next received image
                #
                #  *** NOTES ***
                #  Capturing an image houses images on the camera buffer. Trying
                #  to capture an image that does not exist will hang the camera.
                #
                #  *** LATER ***
                #  Once an image from the buffer is saved and/or no longer
                #  needed, the image must be released in order to keep the
                #  buffer from filling up.
                image_result = cam.GetNextImage()

                #  Ensure image completion
                #
                #  *** NOTES ***
                #  Images can easily be checked for completion. This should be
                #  done whenever a complete image is expected or required.
                #  Further, check image status for a little more insight into
                #  why an image is incomplete.
                if image_result.IsIncomplete():
                    print('Image incomplete with image status %d ...' % image_result.GetImageStatus())

                else:

                    #  Print image information; height and width recorded in pixels
                    #
                    #  *** NOTES ***
                    #  Images have quite a bit of available metadata including
                    #  things such as CRC, image status, and offset values, to
                    #  name a few.
                    width = image_result.GetWidth()
                    height = image_result.GetHeight()
                    print('Grabbed Image %d, width = %d, height = %d' % (i, width, height))

                    #  Convert image to mono 8
                    #
                    #  *** NOTES ***
                    #  Images can be converted between pixel formats by using
                    #  the appropriate enumeration value. Unlike the original
                    #  image, the converted one does not need to be released as
                    #  it does not affect the camera buffer.
                    #
                    #  When converting images, color processing algorithm is an
                    #  optional parameter.
                    image_converted = image_result.Convert(PySpin.PixelFormat_Mono8, PySpin.HQ_LINEAR)

                    # Create a unique filename
                    if device_serial_number:
                        filename = 'Acquisition-%s-%d.jpg' % (device_serial_number, i)
                    else:  # if serial number is empty
                        filename = 'Acquisition-%d.jpg' % i

                    #  Save image
                    #
                    #  *** NOTES ***
                    #  The standard practice of the examples is to use device
                    #  serial numbers to keep images of one device from
                    #  overwriting those of another.
                    image_converted.Save(filename)
                    print('Image saved at %s' % filename)

                    #  Release image
                    #
                    #  *** NOTES ***
                    #  Images retrieved directly from the camera (i.e. non-converted
                    #  images) need to be released in order to keep from filling the
                    #  buffer.
                    image_result.Release()
                    print('')

            except PySpin.SpinnakerException as ex:
                print('Error: %s' % ex)
                return False

        #  End acquisition
        #
        #  *** NOTES ***
        #  Ending acquisition appropriately helps ensure that devices clean up
        #  properly and do not need to be power-cycled to maintain integrity.
        cam.EndAcquisition()
    
    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        return False

    return result


def print_device_info(nodemap):
    """
    This function prints the device information of the camera from the transport
    layer; please see NodeMapInfo example for more in-depth comments on printing
    device information from the nodemap.

    :param nodemap: Transport layer device nodemap.
    :type nodemap: INodeMap
    :returns: True if successful, False otherwise.
    :rtype: bool
    """

    print('*** DEVICE INFORMATION ***\n')

    try:
        result = True
        node_device_information = PySpin.CCategoryPtr(nodemap.GetNode('DeviceInformation'))

        if PySpin.IsAvailable(node_device_information) and PySpin.IsReadable(node_device_information):
            features = node_device_information.GetFeatures()
            for feature in features:
                node_feature = PySpin.CValuePtr(feature)
                print('%s: %s' % (node_feature.GetName(),
                                  node_feature.ToString() if PySpin.IsReadable(node_feature) else 'Node not readable'))

        else:
            print('Device control information not available.')

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        return False

    return result


def run_single_camera(cam):
    """
    This function acts as the body of the example; please see NodeMapInfo example
    for more in-depth comments on setting up cameras.

    :param cam: Camera to run on.
    :type cam: CameraPtr
    :return: True if successful, False otherwise.
    :rtype: bool
    """
    try:
        result = True

        # Retrieve TL device nodemap and print device information
        nodemap_tldevice = cam.GetTLDeviceNodeMap()

        result &= print_device_info(nodemap_tldevice)

        # Initialize camera
        cam.Init()

        # Retrieve GenICam nodemap
        nodemap = cam.GetNodeMap()

        # Acquire images
        result &= acquire_images(cam, nodemap, nodemap_tldevice)

        # Deinitialize camera
        cam.DeInit()

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        result = False

    return result


def main():
    """
    Example entry point; please see Enumeration example for more in-depth
    comments on preparing and cleaning up the system.

    :return: True if successful, False otherwise.
    :rtype: bool
    """

    # Since this application saves images in the current folder
    # we must ensure that we have permission to write to this folder.
    # If we do not have permission, fail right away.
    try:
        test_file = open('test.txt', 'w+')
    except IOError:
        print('Unable to write to current directory. Please check permissions.')
        input('Press Enter to exit...')
        return False

    test_file.close()
    os.remove(test_file.name)

    result = True

    # Retrieve singleton reference to system object
    system = PySpin.System.GetInstance()

    # Get current library version
    version = system.GetLibraryVersion()
    print('Library version: %d.%d.%d.%d' % (version.major, version.minor, version.type, version.build))

    # Retrieve list of cameras from the system
    cam_list = system.GetCameras()

    num_cameras = cam_list.GetSize()

    print('Number of cameras detected: %d' % num_cameras)

    # Finish if there are no cameras
    if num_cameras == 0:

        # Clear camera list before releasing system
        cam_list.Clear()

        # Release system instance
        system.ReleaseInstance()

        print('Not enough cameras!')
        input('Done! Press Enter to exit...')
        return False

    # Run example on each camera
    for i, cam in enumerate(cam_list):

        print('Running example for camera %d...' % i)

        result &= run_single_camera(cam)
        print('Camera %d example complete... \n' % i)

    # Release reference to camera
    # NOTE: Unlike the C++ examples, we cannot rely on pointer objects being automatically
    # cleaned up when going out of scope.
    # The usage of del is preferred to assigning the variable to None.
    del cam

    # Clear camera list before releasing system
    cam_list.Clear()

    # Release system instance
    system.ReleaseInstance()

    input('Done! Press Enter to exit...')
    return result

if __name__ == '__main__':
    main()


Library version: 1.19.0.22
Number of cameras detected: 1
Running example for camera 0...
*** DEVICE INFORMATION ***

DeviceID: 18474805
DeviceSerialNumber: 18474805
DeviceVendorName: FLIR
DeviceModelName: Blackfly S BFS-U3-200S6C
DeviceType: U3V
DeviceDisplayName: FLIR Blackfly S BFS-U3-200S6C
DeviceAccessStatus: ReadWrite
DeviceVersion: 1705.0.124.0
DeviceDriverVersion: PGRUSBCam3.sys : 2.7.3.249
DeviceUserID: 
DeviceIsUpdater: 0
DeviceInstanceId: USB\VID_1E10&PID_4000&MI_00\6&356F58B8&0&0000
DeviceCurrentSpeed: HighSpeed
GUIXMLLocation: Device
GUIXMLPath: Input.xml
GenICamXMLLocation: Device
GenICamXMLPath: 
DeviceU3VProtocol: 1
*** IMAGE ACQUISITION ***

Acquisition mode set to continuous...
Acquiring images...
Device serial number retrieved as 18474805...
Grabbed Image 0, width = 5472, height = 3648
Image saved at Acquisition-18474805-0.jpg

Grabbed Image 1, width = 5472, height = 3648
Image saved at Acquisition-18474805-1.jpg

Grabbed Image 2, width = 5472, height = 3648
Image sav

Done! Press Enter to exit... 


In [3]:
%run ImageFormatControl.py

Library version: 1.19.0.22
Number of cameras detected: 1
Running example for camera 0...
*** DEVICE INFORMATION ***

DeviceID: 18474805
DeviceSerialNumber: 18474805
DeviceVendorName: FLIR
DeviceModelName: Blackfly S BFS-U3-200S6C
DeviceType: U3V
DeviceDisplayName: FLIR Blackfly S BFS-U3-200S6C
DeviceAccessStatus: ReadWrite
DeviceVersion: 1705.0.124.0
DeviceDriverVersion: PGRUSBCam3.sys : 2.7.3.249
DeviceUserID: 
DeviceIsUpdater: 0
DeviceInstanceId: USB\VID_1E10&PID_4000&MI_00\6&356F58B8&0&0000
DeviceCurrentSpeed: HighSpeed
GUIXMLLocation: Device
GUIXMLPath: Input.xml
GenICamXMLLocation: Device
GenICamXMLPath: 
DeviceU3VProtocol: 1

*** CONFIGURING CUSTOM IMAGE SETTINGS *** 

Pixel format set to Mono8...
Height set to 1824...
Error: Spinnaker: GenICam::OutOfRangeException= Value = 1368 must be equal or smaller than Max = 0. : OutOfRangeException thrown in node 'OffsetX' while calling 'OffsetX.SetValue()' (file 'IntegerT.h', line 79) [-2002]
Camera 0 example complete... 



Done! Press Enter to exit... 


In [6]:
%run CounterAndTimer.py

Library version: 1.19.0.22
Number of cameras detected: 1
Running example for camera 0...
*** DEVICE INFORMATION ***

DeviceID: 18474805
DeviceSerialNumber: 18474805
DeviceVendorName: FLIR
DeviceModelName: Blackfly S BFS-U3-200S6C
DeviceType: U3V
DeviceDisplayName: FLIR Blackfly S BFS-U3-200S6C
DeviceAccessStatus: ReadWrite
DeviceVersion: 1705.0.124.0
DeviceDriverVersion: PGRUSBCam3.sys : 2.7.3.249
DeviceUserID: 
DeviceIsUpdater: 0
DeviceInstanceId: USB\VID_1E10&PID_4000&MI_00\6&356F58B8&0&0000
DeviceCurrentSpeed: HighSpeed
GUIXMLLocation: Device
GUIXMLPath: Input.xml
GenICamXMLLocation: Device
GenICamXMLPath: 
DeviceU3VProtocol: 1

Configuring Pulse Width Modulation signal

The duty cycle has been set to 90.4%

The pulse rate has been set to 16.0 Hz

Configuring GPIO strobe output

Configuring Exposure and Trigger

*** IMAGE ACQUISITION ***

Acquisition mode set to continuous...
Acquiring images...
Device serial number retrieved as 18474805...

Grabbed image 0, width = 5472, height = 3

Done! Press Enter to exit... 
